# Go to the end to read

In [2]:
import pandas as pd
import datetime
import os

# Start

In [2]:
bestelbon = pd.read_csv(os.getcwd() + '\\src\\1_raw\\bestelbon_edited.csv', sep=';')

C:\Users\Tim\AppData\Local\Temp\ipykernel_22504\3739138498.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  bestelbon = pd.read_csv(os.getcwd() + '\\src\\1_raw\\bestelbon_edited.csv', sep=';')


In [3]:
bestelbonregels = pd.read_csv(os.getcwd() + '\\src\\1_raw\\bestelbonregels.csv', sep=";")

In [4]:
retourbon = pd.read_csv(os.getcwd() + '\\src\\1_raw\\retourbon_edited.csv', sep=';')

In [5]:
retourbonregels = pd.read_csv(os.getcwd() + '\\src\\1_raw\\retourbonregels.csv', sep=";")

### Add ingangdatum to bestelbonregels

In [6]:
bestelbonregels_merged = bestelbonregels.merge(bestelbon[['Code', 'Ingangdatum huur', 'Order']], left_on='Bestelbon', right_on='Code')

In [7]:
rown = bestelbonregels_merged.index

In [8]:
bestelbonregels_merged['rown'] = rown

### Add einddatum to retourbonregels

In [9]:
retourbonregels_merged = retourbonregels.merge(retourbon[['Code', 'Einddatum huur']], left_on='Retourbon', right_on='Code')

### Add ingangsdatum to retourbonregels on index

In [10]:
endframe = retourbonregels_merged.merge(bestelbonregels_merged[['Ingangdatum huur', 'Aantal', 'rown']], left_on='Bestelbonregel', right_on='rown')

In [11]:
endframe.to_csv('Forecast_articles_with_dates_BUKO.csv')

# Filter one article

In [12]:
AHEKFrame = endframe[endframe['Artikel']=='BUISPOOT']

In [13]:
datetime.datetime.strptime(AHEKFrame['Ingangdatum huur'].values[0],'%d-%m-%Y')

datetime.datetime(2007, 1, 19, 0, 0)

In [14]:
test=pd.to_datetime(AHEKFrame['Ingangdatum huur'],errors='ignore',format='%d-%m-%Y')

In [15]:
test>'01-01-2015'

4          True
5          True
18         True
19         True
36         True
           ... 
2184334    True
2184399    True
2184406    True
2184477    True
2184620    True
Name: Ingangdatum huur, Length: 91365, dtype: bool

In [16]:
AHEKFrame[(AHEKFrame['Ingangdatum huur']>'01-01-2015') & (AHEKFrame['Ingangdatum huur']<'31-12-2015')]

,Code_x,Retourbon,Bestelbonregel,Artikel,Aantal Retour,Aantal manco,Kosten manco,Kortingspercentage manco,Aantal schade,Kosten schade,Kortingspercentage schade,Code_y,Einddatum huur,Ingangdatum huur,Aantal,rown
4,2712,300032.R001,2988,BUISPOOT,8.0,NaN,NaN,NaN,NaN,NaN,...,300032.R001,6-12-2006,19-1-2007,4.0,2988
5,2713,300032.R001,2987,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,300032.R001,6-12-2006,19-1-2007,2.0,2987
18,2902,300052.R001,3150,BUISPOOT,6.0,NaN,NaN,NaN,NaN,NaN,...,300052.R001,11-1-2007,22-1-2007,15.0,3150
19,2903,300052.R001,3151,BUISPOOT,6.0,NaN,NaN,NaN,NaN,NaN,...,300052.R001,11-1-2007,22-1-2007,1.0,3151
58,2946,300131.R001,4020,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,300131.R001,10-1-2007,24-1-2007,2.0,4020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184279,4996117,464754.R009,2969576,BUISPOOT,0.0,8.0,51.1,50.0,NaN,NaN,...,464754.R009,5-2-2021,22-6-2021,1.0,2969576
2184334,5081611,431933.R006,2270206,BUISPOOT,2.0,NaN,NaN,NaN,NaN,NaN,...,431933.R006,13-8-2021,21-11-2019,3.0,2270206
2184406,5139910,432109.R009,2251072,BUISPOOT,18.0,NaN,NaN,NaN,NaN,NaN,...,432109.R009,29-7-2021,18-11-2019,1.0,2251072
2184477,5173770,464067.R002,2969125,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,464067.R002,6-10-2021,24-6-2021,1.0,2969125


## Method 1: explode method

In [17]:
test = AHEKFrame[AHEKFrame['Retourbon']=='460482.R002']

In [18]:
test

,Code_x,Retourbon,Bestelbonregel,Artikel,Aantal Retour,Aantal manco,Kosten manco,Kortingspercentage manco,Aantal schade,Kosten schade,Kortingspercentage schade,Code_y,Einddatum huur,Ingangdatum huur,Aantal,rown
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705


In [19]:
test['date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(test['Ingangdatum huur'], dayfirst=True),
                  pd.to_datetime(test['Einddatum huur'], dayfirst=True))]

C:\Users\Tim\AppData\Local\Temp\ipykernel_22504\3224146269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date'] = [pd.date_range(s, e, freq='d') for s, e in


In [20]:
test

,Code_x,Retourbon,Bestelbonregel,Artikel,Aantal Retour,Aantal manco,Kosten manco,Kortingspercentage manco,Aantal schade,Kosten schade,Kortingspercentage schade,Code_y,Einddatum huur,Ingangdatum huur,Aantal,rown,date
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,"DatetimeIndex(['2021-04-07', '2021-04-08', '20..."


In [21]:
test = test.explode('date')

In [22]:
test

,Code_x,Retourbon,Bestelbonregel,Artikel,Aantal Retour,Aantal manco,Kosten manco,Kortingspercentage manco,Aantal schade,Kosten schade,Kortingspercentage schade,Code_y,Einddatum huur,Ingangdatum huur,Aantal,rown,date
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-04-07
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-04-08
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-04-09
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-04-10
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-04-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-07-25
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-07-26
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-07-27
2184399,5139902,460482.R002,2867705,BUISPOOT,4.0,NaN,NaN,NaN,NaN,NaN,...,460482.R002,29-7-2021,7-4-2021,1.0,2867705,2021-07-28


## Method 2: count on date method

creating a new merged bestel_bon processed and retourbon _processed files.


In [3]:
import numpy as np
import pandas as pd
import csv

In [51]:
regels_file=r'C:\Users\Amey.RISE3D\OneDrive - RISE3D\Documents\BUKO_FORECASTING\Raw_data_van_Tim\BISAS Data Ruw\bestelbonregels.csv'
df_bestelbonregels= pd.read_csv(regels_file, sep=";")


In [12]:
bon_file=r'C:\Users\Amey.RISE3D\OneDrive - RISE3D\Documents\BUKO_FORECASTING\Raw_data_van_Tim\BISAS Data Ruw\bestelbon.csv'
df_bestelbon_all=pd.read_csv(bon_file,sep=";",on_bad_lines='warn')

b'Skipping line 58210: expected 10 fields, saw 11\n'
b'Skipping line 79246: expected 10 fields, saw 11\nSkipping line 93228: expected 10 fields, saw 11\nSkipping line 94143: expected 10 fields, saw 11\nSkipping line 126370: expected 10 fields, saw 11\n'
b'Skipping line 181323: expected 10 fields, saw 11\nSkipping line 185917: expected 10 fields, saw 11\n'
b'Skipping line 198362: expected 10 fields, saw 11\nSkipping line 219557: expected 10 fields, saw 11\nSkipping line 242387: expected 10 fields, saw 11\n'
b'Skipping line 276111: expected 10 fields, saw 12\nSkipping line 277617: expected 10 fields, saw 12\nSkipping line 327669: expected 10 fields, saw 11\n'


In [38]:
import csv
df_bestelbon=pd.read_csv(bon_file,sep=";",on_bad_lines='warn',usecols=[0,1,6], quoting=csv.QUOTE_NONE)

In [44]:
df_bestelbonregels_merged = df_bestelbonregels.merge(df_bestelbon, left_on='Bestelbon', right_on='Code')

In [41]:
b_order_merge=df_bestelbonregels_merged['Order']


In [49]:
a=np.where(b_order_merge==395853)[0]
# a=np.where(b_order==433777)[0]
b_order_merge[a]

1146259    395853.0
1146260    395853.0
1146261    395853.0
1146262    395853.0
1146263    395853.0
1146264    395853.0
1146265    395853.0
1146266    395853.0
1146267    395853.0
1146268    395853.0
1146269    395853.0
1146270    395853.0
1146271    395853.0
Name: Order, dtype: float64

In [45]:
bb_merged_file=r'C:\Users\Amey.RISE3D\OneDrive - RISE3D\Documents\BUKO_FORECASTING\Raw_data_van_Tim\BISAS Data Ruw\bestel_processed.csv'
df_bestelbonregels_merged.to_csv(bb_merged_file)

retour bon

In [8]:
regels_file=r'/home/amey/Projects/BUKO_Forecasting/Raw_data_van_Tim/BISAS Data Ruw/retourbonregels.csv'

df_retourbonregels= pd.read_csv(regels_file, sep=";")


In [9]:
ret_file=r'/home/amey/Projects/BUKO_Forecasting/Raw_data_van_Tim/BISAS Data Ruw/retourbon.csv'
# df_retourbon_all=pd.read_csv(ret_file,sep=";",on_bad_lines='warn')
df_retourbon=pd.read_csv(ret_file,sep=";",on_bad_lines='warn',usecols=[0,1,6], quoting=csv.QUOTE_NONE)

In [10]:
df_retourbonregels_merged = df_retourbonregels.merge(df_retourbon, left_on='Retourbon', right_on='Code')

In [ ]:
rb_merged_file=r'/home/amey/Projects/BUKO_Forecasting/Raw_data_van_Tim/BISAS Data Ruw/retour_processed.csv'
df_retourbonregels_merged.to_csv(rb_merged_file)

In [14]:
df_retourbonregels_merged

,Code_x,Retourbon,Bestelbonregel,Artikel,Aantal Retour,Aantal manco,Kosten manco,Kortingspercentage manco,Aantal schade,Kosten schade,Kortingspercentage schade,Code_y,Order,Einddatum huur
0,2321,300026.R001,2622,DRANGHEK,60.0,NaN,NaN,NaN,NaN,NaN,...,300026.R001,300026.0,02-01-2007
1,2322,300033.R001,2667,991,1.0,NaN,NaN,NaN,NaN,NaN,...,300033.R001,300033.0,03-01-2007
2,2341,300024.R001,2679,VERVALLEN,1.0,NaN,NaN,NaN,NaN,NaN,...,300024.R001,300024.0,03-01-2007
3,2711,300032.R001,2986,AHEK,6.0,NaN,NaN,NaN,NaN,NaN,...,300032.R001,300032.0,06-12-2006
4,2712,300032.R001,2988,BUISPOOT,8.0,NaN,NaN,NaN,NaN,NaN,...,300032.R001,300032.0,06-12-2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3072917,5397479,525692.R001,4597255,GRONDPOT,2.0,0.0,NaN,NaN,0.0,NaN,...,525692.R001,525692.0,07-02-2022
3072918,5397480,527339.R001,4635502,E01-60,3.0,NaN,NaN,NaN,NaN,NaN,...,527339.R001,527339.0,11-02-2022
3072919,5397481,527339.R001,4635500,P3,3.0,NaN,NaN,NaN,NaN,NaN,...,527339.R001,527339.0,11-02-2022
3072920,5397482,527339.R001,4635503,GRONDPOT,3.0,NaN,NaN,NaN,NaN,NaN,...,527339.R001,527339.0,11-02-2022


Order _processing.


In [9]:
order_file=r'/home/amey/Projects/BUKO_Forecasting/Raw_data_van_Tim/BISAS Data Ruw/order.csv'
df_order=pd.read_csv(order_file,header=0,usecols=[0,2],encoding_errors='backslashreplace',encoding='utf-8',sep=';',on_bad_lines='warn')

/tmp/ipykernel_3173202/1252038322.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_order=pd.read_csv(order_file,header=0,usecols=[0,2],encoding_errors='backslashreplace',encoding='utf-8',sep=';',on_bad_lines='warn')


In [11]:
df_order.to_csv(r'/home/amey/Projects/BUKO_Forecasting/Raw_data_van_Tim/BISAS Data Ruw/order_edited.csv')

In [1]:
import datetime
datestr='08-01-2007'
format='%d-%m-%Y'
dt_obj=datetime.datetime.strptime(datestr,format)